# Intro

The OECD database is composed of hundreds (if not thousands) of tables that focus on specific economic and social variables. These variables are attirbutable to individual countries, and are generally recorded over time, making them a time series. 

For the purposes of this analysis, we are only interested in the latest year's data point, which would reflect the most recent characterization of each country's social and economic situation. 

However, given that we have years of data for each country, there is also the potential to use the change in the variable as a feature of our modeling. For example, rapid improvement in immunization coverage for a country would imply effective health education and could represent a more accurate picture of how the country is better educating the public about a new pandemic versus a country with a higher, but stagnant, immunization record. 

For now, we are combining the various OECD tables on the 'country' key so that we can merge it with COVID data.

Data transformation to follow.

# Loading and Cleaning Data

Here we load and clean csv data taken from the OECD database. This includes datasets on the following topics:
- Education
- Government Debt
- Immunization
- Demographics / Population
- Tourism
- Wealth Distribution
- Life Expectancy

For each dataset, there are multiple columns we are not interested in. Furthermore, for the columns we do want, there are different enteries for that variable, so we need to filter on what we are looking for. 

The tables and column names are not standardized, so each csv file will have to be cleaned and prepared individually. 

### Load Packages

In [3]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
import ssl
import json
import requests

ssl._create_default_https_context = ssl._create_unverified_context

## OECD


### Education

We hypothesize that a country's education should play a key role in the success rate of the country's approach to COVID. Specifically, a more highly educated country should, in theory, have a more effective approach to dealing with the virus, and should have a populace that better understands public health terminiology, what viruses are, and would be more willing to take the pandemic seriously. 

So we will focus on the share of population of countries that have a tertiary education level. This is defined on wikipedia as:

*Tertiary education, also referred to as third-level, third-stage or post-secondary education, is the educational level following the completion of secondary education. The World Bank, for example, defines tertiary education as including universities as well as trade schools and colleges.*

In [26]:
url_edu = 'https://raw.githubusercontent.com/pvai-umich/SIADS591/master/Data/OECD_Education_Statistics.csv'
dfraw_edu = pd.read_csv(url_edu)


In [27]:
columns_to_use = ['COUNTRY', 'Country', 'Gender', 'ISCED 2011 A education level', 'Reference Period', 'Measure', 'Value']

# Only select the columns we want to use
df_edu = dfraw_edu[columns_to_use]

# filter some of the columns to include the variables we want to see
df_edu = df_edu[df_edu['ISCED 2011 A education level'] == "Tertiary education"]
df_edu = df_edu[df_edu['Gender'] == "Total"]
df_edu = df_edu[df_edu['Measure'] == "Value"]

# Choose the latest datapoint for each country
df_edu = df_edu.sort_values('Reference Period').groupby('Country').tail(1)

# This final dataframe contains the share of each country's population that has a tertiary education. 
df_edu.head()


,COUNTRY,Country,Gender,ISCED 2011 A education level,Reference Period,Measure,Value
3244,CHN,China (People's Republic of),Total,Tertiary education,2010.0,Value,9.681174
3302,IND,India,Total,Tertiary education,2011.0,Value,10.595189
2846,SAU,Saudi Arabia,Total,Tertiary education,2016.0,Value,23.623922
2772,CHL,Chile,Total,Tertiary education,2017.0,Value,25.168179
2757,RUS,Russia,Total,Tertiary education,2017.0,Value,56.700150


In [28]:
# But lets reduce this table down to what we'll be combining together later.
# A simple country-variable table.

df_edu = df_edu[['COUNTRY', 'Country', 'Value']]
df_edu.columns = ['ISO', 'Country', 'Tertiary_Education_Pct']
df_edu.head()

,ISO,Country,Tertiary_Education_Pct
3244,CHN,China (People's Republic of),9.681174
3302,IND,India,10.595189
2846,SAU,Saudi Arabia,23.623922
2772,CHL,Chile,25.168179
2757,RUS,Russia,56.700150


### Debt

The Debt dataset from the OECD has various government-level debt statistics. Our focus here will be on the total debt-to-GDP ratio, as it helps us look at a normalized measure, rather than needing to adjust absolute debt levels across different sized countries.

Our hypothesis is a little mixed. On the surface, one would expect that a heavily indebted country is not wealthy and might have to skimp on health care spending at a time of a pandemic. Or conversely, wouldn't have room to borrow more. However, anecdotally, Japan is one of the most indebted countries in the World, but is highly developed, has a great health care system, and can easily increase its debt levels. This might be a difficult variable to gain insights on.

In [29]:
# Load the Debt Data
url_debt = 'https://raw.githubusercontent.com/pvai-umich/SIADS591/master/Data/OECD_Government_Debt.csv'
dfraw_debt = pd.read_csv(url_debt)

In [30]:
# Choose the latest data point for each country
df_debt = dfraw_debt.sort_values('Time period').groupby('Country').tail(1)

# Choose the columns we want and rename them.
df_debt = df_debt[['COU', 'Country', 'Value']]
df_debt.columns = ['ISO', 'Country', 'Debt_to_GDP']

# Check to make sure it looks ok
df_debt.head()

,ISO,Country,Debt_to_GDP
208,JPN,Japan,183.530
278,NZL,New Zealand,30.450
264,NLD,Netherlands,51.845
320,PRT,Portugal,87.962
27,AUT,Austria,65.754


### Immunization

OECD also has a slew of health care data. It is not a health care organization however, so we don't want to rely too heavily on medical statistics from this source, however, they do have useful macro level information that is insightful. 

One of these variables is the percentage of individuals over 65 that are immunized for the flu. Unfortunately this isn't a statistic they have for the full population, but it is still a good proxy for how open a country is to public health initiatives, and how seriously they take flu-like diseases. 

Our hypothesis would be that the higher the level of immunization, the more successful a country is at combatting COVID.


In [31]:
# Load Immunization Data
url_imm = 'https://raw.githubusercontent.com/pvai-umich/SIADS591/master/Data/OECD_Immunization_Statistics.csv'
dfraw_imm = pd.read_csv(url_imm)


In [32]:
# Choose the latest data point for each country
df_imm = dfraw_imm.sort_values('Year').groupby('Country').tail(1)

# Choose the columns we want and rename them.
df_imm = df_imm[['COU', 'Country', 'Value']]
df_imm.columns = ['ISO', 'Country', 'Pct_65_Immunized_Flu']

# Check to make sure it looks ok
df_imm.head()

,ISO,Country,Pct_65_Immunized_Flu
6,AUS,Australia,74.6
351,CHE,Switzerland,46.0
8,AUT,Austria,20.3
271,POL,Poland,9.7
224,MEX,Mexico,82.3


### Population / Demographics

OECD offers a slew of population and demographic data for global countries. Our variable of interest is regarding age demographics of countries. Specifically we will look at the Old Age Dependency Ratio. This is the ratio of a country's old-age population (>65 years) to the rest of the population. Our hypothesis is there is a positive correlation between COVID success and this ratio


In [33]:
# Load the Demographics Data
url_pop = 'https://raw.githubusercontent.com/pvai-umich/SIADS591/master/Data/OECD_Population_Statistics.csv'
dfraw_pop = pd.read_csv(url_pop)

In [34]:
# Create an old age ratio dataframe
df_oldage = dfraw_pop[dfraw_pop['AGE'] == "65_OVER_SHARE"]
df_oldage = df_oldage[df_oldage['Sex'] == "Total"]

# Choose the latest data point for each country
df_oldage = df_oldage.sort_values('Time').groupby('Country').tail(1)

# Choose the columns we want and rename them.
df_oldage = df_oldage[['LOCATION', 'Country', 'Value']]
df_oldage.columns = ['ISO', 'Country', 'Old_Age_Ratio']

# Check to make sure it looks ok
df_oldage.head()

,ISO,Country,Old_Age_Ratio
58757,EU28,European Union (28 countries),19.849
58715,G20,G20,11.037
58673,WLD,World,8.921
57623,TUR,Turkey,8.649
58589,SVN,Slovenia,19.670


In [35]:
# Also retain the total population of each country so we can 
# calc death per capita later on

df_population = dfraw_pop[dfraw_pop['Age'] == "Total"]
df_population = df_population[df_population['Sex'] == "Total"]

df_population = df_population.sort_values('Time').groupby('Country').tail(1)

# Choose the columns we want and rename them.
df_population = df_population[['LOCATION', 'Country', 'Value']]
df_population.columns = ['ISO', 'Country', 'Population']

# Check to make sure it looks ok
df_population.head()


,ISO,Country,Population
2309,EU28,European Union (28 countries),5.131792e+08
2267,G20,G20,4.809945e+09
2225,WLD,World,7.632819e+09
1175,TUR,Turkey,8.140721e+07
2141,SVN,Slovenia,2.070050e+06


### Tourism

In [36]:
url_tour = 'https://raw.githubusercontent.com/pvai-umich/SIADS591/master/Data/OECD_Tourism_Statistics.csv'
dfraw_tour = pd.read_csv(url_tour)


In [37]:
# We'll create two separate dataframes from the tourism table
# Tourism as % of GDP (Do a lot of people come to the country)
# and Tourism as % of Total Employment (do a lot of people interact with foreigners)

# Create Tourism as % of GDP dataframe
df_tour_gdp = dfraw_tour[dfraw_tour['KEY_IND_PC'] == "KEYIND_TOURGDPPC"]

# Choose the latest data point for each country
df_tour_gdp = df_tour_gdp.sort_values('Year').groupby('Country').tail(1)

# Choose the columns we want and rename them.
df_tour_gdp = df_tour_gdp[['COUNTRY', 'Country', 'Value']]
df_tour_gdp.columns = ['ISO', 'Country', 'Tourism_Pct_GDP']

# Check to make sure it looks ok
df_tour_gdp.head()



,ISO,Country,Tourism_Pct_GDP
564,MLT,Malta,6.129
119,IND,India,3.759
108,EGY,Egypt,4.258
115,EST,Estonia,5.428
66,POL,Poland,1.175


In [38]:
# Create Toursim employment as % of Total
df_tour_emp = dfraw_tour[dfraw_tour['KEY_IND_PC'] == "KEYIND_TOUREMPDIRECTPC"]

# Choose the latest data point for each country
df_tour_emp = df_tour_emp.sort_values('Year').groupby('Country').tail(1)

# Choose the columns we want and rename them.
df_tour_emp = df_tour_emp[['COUNTRY', 'Country', 'Value']]
df_tour_emp.columns = ['ISO', 'Country', 'Tourism_Pct_Employment']

# Check to make sure it looks ok
df_tour_emp.head()

,ISO,Country,Tourism_Pct_Employment
387,ARG,Argentina,10.17
460,IND,India,5.31
465,IDN,Indonesia,8.37
577,PER,Peru,3.82
282,ITA,Italy,8.30


### Wealth Distribution

In [39]:
url_wealth = 'https://raw.githubusercontent.com/pvai-umich/SIADS591/master/Data/OECD_Wealth_Distribution_Statistics.csv'
dfraw_wealth = pd.read_csv(url_wealth)

In [40]:
# Create two dataframes from the wealth distirbution table. The focus is on
# inequality is the focus of the data.
# Share of top 10% wealth (how rich are the super-rich)
# Share of bottom 60% of wealth (how poor is the bottom half)
dfraw_wealth.head()

# First, create the top 10pct dataframe
df_top_10pct_share = dfraw_wealth[dfraw_wealth['Variable'] == 'Share of top 10% of wealth']

# Choose the latest data point for each country
df_top_10pct_share = df_top_10pct_share.sort_values('Time').groupby('Country').tail(1)

# Choose the columns we want and rename them.
df_top_10pct_share = df_top_10pct_share[['COUNTRY', 'Country', 'Value']]
df_top_10pct_share.columns = ['ISO', 'Country', 'Top_10pct_Share']

# Check to make sure it looks ok
df_top_10pct_share.head()


,ISO,Country,Top_10pct_Share
268,ESP,Spain,45.577541
752,PRT,Portugal,52.127502
487,IRL,Ireland,53.792400
325,FIN,Finland,45.231823
288,EST,Estonia,55.713390


In [41]:
# Create the bottom 60% dataframe

df_bottom_60pct_share = dfraw_wealth[dfraw_wealth['Variable'] == 'Share of bottom 60% of wealth']

# Choose the latest data point for each country
df_bottom_60pct_share = df_bottom_60pct_share.sort_values('Time').groupby('Country').tail(1)

# Choose the columns we want and rename them.
df_bottom_60pct_share = df_bottom_60pct_share[['COUNTRY', 'Country', 'Value']]
df_bottom_60pct_share.columns = ['ISO', 'Country', 'Bottom_60pct_Share']

# Check to make sure it looks ok
df_bottom_60pct_share.head()

,ISO,Country,Bottom_60pct_Share
252,ESP,Spain,18.664709
280,EST,Estonia,12.786300
736,PRT,Portugal,12.422934
479,IRL,Ireland,7.184053
309,FIN,Finland,13.552249


### Life Expectancy


In [42]:
url_life_exp = 'https://raw.githubusercontent.com/pvai-umich/SIADS591/master/Data/OECD_Life_Expectancy.csv'
dfraw_life_exp = pd.read_csv(url_life_exp)

In [43]:
# There's only one variable here, so we just need to get the last available date
# and then choose the columns we want 

# Choose the latest data point for each country
df_life_exp = dfraw_life_exp.sort_values('Year').groupby('Country').tail(1)

# Choose the columns we want and rename them.
df_life_exp = df_life_exp[['COU', 'Country', 'Value']]
df_life_exp.columns = ['ISO', 'Country', 'Life_Expectancy']

# Check to make sure it looks ok
df_life_exp.head()

,ISO,Country,Life_Expectancy
133,JPN,Japan,84.2
368,LVA,Latvia,74.9
71,FRA,France,82.8
377,CRI,Costa Rica,80.4
386,LTU,Lithuania,75.8


### Health Expenditures

This is the total expenditures on health care as a % of GDP

In [44]:
url_health_exp = 'https://raw.githubusercontent.com/pvai-umich/SIADS591/master/Data/OECD_Health_Exp.csv'
dfraw_health_exp = pd.read_csv(url_health_exp)

In [77]:
# Filter some of the varibles to what we want
df_health_exp = dfraw_health_exp[dfraw_health_exp['Financing scheme'] == 'All financing schemes']
df_health_exp = df_health_exp[dfraw_health_exp['Function'] == 'Current expenditure on health (all functions)']
df_health_exp = df_health_exp[dfraw_health_exp['Provider'] == 'All providers']
df_health_exp = df_health_exp[dfraw_health_exp['Measure'] == 'Share of gross domestic product']

# Choose the latest data point for each country
df_health_exp = df_health_exp.sort_values('Year').groupby('Country').tail(1)

# Choose the columns we want and rename them.
df_health_exp = df_health_exp[['LOCATION', 'Country', 'Value']]
df_health_exp.columns = ['ISO', 'Country', 'Health_Expenditure']

# Check to make sure it looks ok
df_health_exp.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


,ISO,Country,Health_Expenditure
27874,CHN,China (People's Republic of),5.047
27917,IND,India,3.603
26179,ZAF,South Africa,8.113
24973,BRA,Brazil,9.423
25495,IDN,Indonesia,2.988


### Tobacco Usage

This is the % of population over the age of 15 that uses tobacco

In [46]:
url_tobacco = 'https://raw.githubusercontent.com/pvai-umich/SIADS591/master/Data/OECD_Tobacco.csv'
dfraw_tobacco = pd.read_csv(url_tobacco)

In [48]:
df_tobacco = dfraw_tobacco[dfraw_tobacco['Measure'] == '% of population aged 15+ who are daily smokers']

# Choose the latest data point for each country
df_tobacco = df_tobacco.sort_values('Year').groupby('Country').tail(1)

# Choose the columns we want and rename them.
df_tobacco = df_tobacco[['COU', 'Country', 'Value']]
df_tobacco.columns = ['ISO', 'Country', 'Tobacco_Users']

# Check to make sure it looks ok
df_tobacco.head()

,ISO,Country,Tobacco_Users
2860,COL,Colombia,13.0
2923,LVA,Latvia,24.1
67,AUT,Austria,24.3
816,HUN,Hungary,25.8
2833,SVN,Slovenia,18.9


## COVID-19

All attempts to load json via web is throwing an SSL error. We will load locally.

-Method 1
-Request fails unless we provide a user-agent
api_response = requests.get('https://thevirustracker.com/timeline/map-data.json', headers={"User-Agent": "Chrome"})
covid_stats = api_response.json()

-Method 2
response = urlopen('https://thevirustracker.com/timeline/map-data.json')
json_data = response.read().decode('utf-8', 'replace')
df = json.loads(json_data)

### Covid-19 Daily Stats by Country

In [ ]:
#Importing COVID19 Global Data
cf = 'covid_full.json'
with open(cf, 'r') as f:
    doc = json.load(f)

In [ ]:
#converting to pandas data frame
covid_daily_stats=pd.DataFrame(doc)

#changing the types of the columns
covid_daily_stats['cases']=covid_daily_stats['cases'].astype('int')
covid_daily_stats['deaths']=covid_daily_stats['deaths'].astype('int')
#covid_daily_stats['recovered']=covid_daily_stats['recovered'].astype('int')

#renaming columns for future merge
covid_daily_stats.rename(columns={'countrycode':'Alpha-2 code'}, inplace=True)

covid_daily_stats.sample(10)

,Alpha-2 code,date,cases,deaths,recovered
27707,BH,7/31/20,40982,147,25178
1641,BG,4/18/20,878,41,3
32820,QA,8/26/20,117742,194,94903
11503,MX,4/02/20,1510,50,4
17476,ZA,5/11/20,10652,206,2
32290,IQ,8/16/20,176931,5860,36252
31242,DJ,8/20/20,5374,59,4621
34604,CR,9/04/20,45680,469,1810
29886,YE,8/12/20,1841,528,591
4498,DO,5/15/20,11739,424,0


In [ ]:
covid_daily_stats.dtypes

Alpha-2 code    object
date            object
cases            int64
deaths           int64
recovered       object
dtype: object

### Covid-19 Global Stats

In [ ]:
#Importing COVID19 Global Data
gs= 'global_stats.json'
with open(gs, 'r') as g:
    doc1 = json.load(g)
covid_global_stats=pd.DataFrame(doc1).transpose()
covid_global_stats.rename(columns={0:'stats'}, inplace=True)
covid_global_stats.head()

,stats
total_cases,23164025
total_recovered,15742473
total_unresolved,5940153
total_deaths,804028
total_new_cases_today,0


### Covid-19 Total Stats by Country

In [ ]:
#Group daily covid data by country.. pull out max values
cts=covid_daily_stats.groupby(['Alpha-2 code']).max()
cts['death_ratio']=cts['deaths']/cts['cases']

#Sort values by Death Ratio
cts.sort_values(by='death_ratio', ascending=False)
cts.drop(columns=['date'],inplace=True)

cts.head(10)

,cases,deaths,recovered,death_ratio
Alpha-2 code,,,,
AD,1301,53,800,0.040738
AE,75981,393,7,0.005172
AF,38544,1420,5,0.036841
AG,95,3,23,0.031579
AL,10704,322,67,0.030082
AM,45152,905,31,0.020043
AO,3092,126,117,0.040750
AR,500034,10457,72,0.020913
AT,30583,747,9,0.024425


In [ ]:
cts.filter(like='US', axis=0)

,cases,deaths,recovered,death_ratio
Alpha-2 code,,,,
US,6358247,190727,924148,0.029997


### ISO Translation

In [ ]:
ISO=pd.read_excel('ISO_CODES.xlsx')
ISO.head()

,Country,Alpha-2 code,Alpha-3 code,Numeric
0,Afghanistan,AF,AFG,4
1,Albania,AL,ALB,8
2,Algeria,DZ,DZA,12
3,American Samoa,AS,ASM,16
4,Andorra,AD,AND,20


In [ ]:
cts=cts.merge(ISO,on='Alpha-2 code')
cts.columns

Index(['Alpha-2 code', 'cases', 'deaths', 'recovered', 'death_ratio',
       'Country', 'Alpha-3 code', 'Numeric'],
      dtype='object')

## Combining the OECD Data

For now, we are combining the various OECD tables on the 'country' key so that we can merge it with COVID data.

At this point, there are XXXXX variables we are looking to 

In [86]:
# Let's keep a running list of the dataframe names. Will make it easier
# to run future analysis if we can just iterate on this list of df names rather
# than including each one individually in multiple spots
# So if a new variable is added up top, append it to this list, and the rest
# of the analysis is good to go.
oecd_df_list = list()
oecd_df_list.append(df_edu)
oecd_df_list.append(df_debt)
oecd_df_list.append(df_oldage)
#oecd_df_list.append(df_tour_gdp)
oecd_df_list.append(df_tour_emp)
oecd_df_list.append(df_top_10pct_share)
#oecd_df_list.append(df_bottom_60pct_share)
oecd_df_list.append(df_life_exp)
oecd_df_list.append(df_population)
oecd_df_list.append(df_tobacco)
oecd_df_list.append(df_health_exp)

# Doing a quick check of the shapes of each data frame.
# it looks like some are smaller than others. Will need to decide what countries to keep later.
#for i in oecd_df_list:
  #print(i.shape)

# Let's merge them all together
df_oecd = pd.DataFrame(columns=['ISO', 'Country'])

for df in oecd_df_list:
  df_oecd = df_oecd.merge(df, on=['ISO', 'Country'], how='outer') 

# and we'll drop the NA rows so that we have a complete data set.
# We end up dropping quite a few countries, but the final dataframe
# has 23 which seems enough?
# One other option would be to drop the top_10_pct variable. This would increase
# the country size to 30, which is significant, but I like the idea of a 
# having an inequality statistic. Can think about it going forward.

df_oecd = df_oecd.dropna(axis = 0)

# Let's check it out!
df_oecd.head()

,ISO,Country,Tertiary_Education_Pct,Debt_to_GDP,Old_Age_Ratio,Tourism_Pct_Employment,Top_10pct_Share,Life_Expectancy,Population,Tobacco_Users,Health_Expenditure
3,CHL,Chile,25.168179,9.185,11.547,6.461,57.710564,80.6,18751405.0,24.5,9.102
6,GRC,Greece,31.739904,147.839,21.894,9.974,42.423824,81.9,10725886.0,35.0,7.791
8,PRT,Portugal,24.984337,87.962,21.672,9.810,52.127502,81.4,10283822.0,16.8,9.563
10,HUN,Hungary,25.101177,73.898,19.143,9.590,48.477074,76.2,9767600.0,25.8,6.360
11,GBR,United Kingdom,45.781677,85.535,18.312,4.670,51.988647,81.3,66435550.0,16.6,10.254


### Master Combination

In [80]:
df_merged = cts.merge(df_oecd, left_on='Alpha-3 code', right_on='ISO')

NameError: ignored

In [ ]:
df_merged.sample(10)

,Alpha-2 code,cases,deaths,recovered,death_ratio,Country_x,Alpha-3 code,Numeric,ISO,Country_y,Tertiary_Education_Pct,Debt_to_GDP,Old_Age_Ratio,Tourism_Pct_GDP,Tourism_Pct_Employment,Top_10pct_Share,Bottom_60pct_Share,Life_Expectancy
26,IE,30164,1781,5,0.059044,Ireland,IRL,372,IRL,Ireland,46.935135,60.703,13.864,NaN,10.279,53.792400,7.184053,82.3
21,GB,355219,41594,8,0.117094,United Kingdom of Great Britain and Northern I...,GBR,826,GBR,United Kingdom,45.781677,85.535,18.312,NaN,4.670,51.988647,11.807471,81.3
9,CN,90087,4733,9298,0.052538,China,CHN,156,CHN,China (People's Republic of),9.681174,NaN,11.194,NaN,NaN,NaN,NaN,76.8
14,DE,256433,9342,9211,0.036431,Germany,DEU,276,DEU,Germany,29.065134,44.403,21.466,NaN,4.796,59.758945,6.456923,81.0
24,HU,9715,628,7,0.064642,Hungary,HUN,348,HUN,Hungary,25.101177,73.898,19.143,NaN,9.590,48.477074,15.445245,76.2
32,KR,21588,344,9,0.015935,Korea (the Republic of),KOR,410,KOR,Korea,49.008511,31.935,14.294,NaN,NaN,NaN,17.694309,82.7
34,LT,3163,86,7,0.027189,Lithuania,LTU,440,LTU,Lithuania,41.654594,NaN,19.706,NaN,4.900,NaN,NaN,75.8
16,EE,2585,69,8,0.026692,Estonia,EST,233,EST,Estonia,41.130722,3.227,19.676,5.428,4.300,55.713390,12.786300,78.4
43,PE,696190,30123,53,0.043268,Peru,PER,604,PER,Peru,NaN,NaN,NaN,3.900,3.820,NaN,NaN,NaN
31,JP,73266,1412,9,0.019272,Japan,JPN,392,JPN,Japan,51.928062,183.530,28.137,1.961,9.610,41.016430,17.749653,84.2


In [ ]:
df_merged.count()

Alpha-2 code              57
cases                     57
deaths                    57
recovered                 57
death_ratio               57
Country_x                 57
Alpha-3 code              57
Numeric                   57
ISO                       57
Country_y                 57
Tertiary_Education_Pct    46
Debt_to_GDP               34
Old_Age_Ratio             52
Tourism_Pct_GDP           24
Tourism_Pct_Employment    46
Top_10pct_Share           27
Bottom_60pct_Share        28
Life_Expectancy           44
dtype: int64

In [ ]:
print(len(df_oecd))
print(len(cts))

62
185
